In [ ]:
"""setup"""
import pandas as pd
import pytest
import sys
sys.path.append('../..')
from altair_widgets import interact_with
from altair import load_dataset

In [ ]:
"""test_input_fail"""
d = [{'a': 1, 'b': 2, 'c': 4}]*4

with pytest.raises(ValueError):
    interact_with(d)

In [ ]:
"""test_input"""
df = pd.DataFrame(d)
interact_with(df, show=False)

In [ ]:
"""test_basic_config"""
df = load_dataset('iris')
df.head()
w = interact_with(df, show=False)
w.settings['encodings'] = [{'encoding': 'x', 'column': 'petalWidth'},
                           {'encoding': 'y', 'column': 'petalLength'},
                           {'encoding': 'color', 'column': 'species'}]
w.plot(show=False)

In [ ]:
"""test_generates_same_chart_as_altair"""
import altair
from altair import Chart

c = Chart(df).mark_point().encode(
    x='petalWidth', y='petalLength', color='species')

assert type(w.chart) is altair.api.Chart
assert c.to_dict() == w.chart.to_dict()

In [ ]:
"""test_bg_color"""
w = interact_with(df, show=False)
w.settings = {'mark': {'mark': 'mark_text', 'applyColorToBackground': True},
              'encodings': [{'encoding': 'row', 'column': 'species'},
                            {'encoding': 'color', 'column': 'petalLength'},
                            {'encoding': 'text', 'column': ' '}]
              }
w.plot(show=False)
c = Chart(df).mark_text(applyColorToBackground=True).encode(
    row='species',
    color='petalLength',
    text=' ')
    
assert c.to_dict() == w.chart.to_dict()

In [ ]:
"""test_log"""
w = interact_with(df, show=False)
w.settings['encodings'] = [{'encoding': 'x', 'column': 'petalWidth', 'scale': 'log'},
                           {'encoding': 'y', 'column': 'petalLength'},
                           {'encoding': 'color', 'column': 'species'}]
w.plot(show=False)
c = Chart(df).mark_point().encode(
    x=altair.X('petalWidth', scale=altair.Scale(type='log')),
    y='petalLength', color='species')
    
assert c.to_dict() == w.chart.to_dict()

In [ ]:
"""test_update"""
class Event:
    def __init__(self, row, title, value):
        self.row = row
        self.title = title
        self.value = value

w = interact_with(df, show=False)
w.settings['encodings'] = [{'encoding': 'x', 'column': 'petalWidth', 'scale': 'log'},
                           {'encoding': 'y', 'column': 'petalLength'},
                           {'encoding': 'color', 'column': 'species'}]
old = 'linear'
event = {'owner': Event(1, 'scale', old), 'old': old, 'new': 'log'}
w._update(event)
new_settings = [{'encoding': 'x', 'column': 'petalWidth', 'scale': 'log'},
                {'encoding': 'y', 'column': 'petalLength', 'scale': 'log'},
                {'encoding': 'color', 'column': 'species'}]

assert new_settings == w.settings['encodings']

In [ ]:
"""test_adding_dimension"""
w = interact_with(df, show=False)
w.settings['encodings'] = [{'encoding': 'x', 'column': 'petalWidth'},
                           {'encoding': 'y', 'column': 'petalLength'},
                           {'encoding': 'color', 'column': 'species'}]
w._add_dim(None)
w.settings['encodings'][-1]['encoding'] = 'size'
w.settings['encodings'][-1]['column'] = 'sepalLength'
w.plot(w.settings)

c = Chart(df).mark_point().encode(
    x='petalWidth', y='petalLength', color='species', size='sepalLength'
)

assert w.chart.to_dict() == c.to_dict()

In [ ]:
"""test_ndims"""
w = interact_with(df, ndims=1, show=False)
assert len(w.settings['encodings']) == 1